In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_taxi_trips
(CONSTRAINT valid_distance EXPECT (trip_distance > 0.0) ON VIOLATION DROP ROW)
AS SELECT *
FROM STREAM(samples.nyctaxi.trips);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_flagged_rides 
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  pickup_zip as zip, 
  fare_amount, trip_distance
FROM
  STREAM(LIVE.bronze_taxi_trips)
WHERE ((pickup_zip = dropoff_zip AND fare_amount > 50) OR
       (trip_distance < 5 AND fare_amount > 50));

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW silver_weekly_stats
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  AVG(fare_amount) as avg_amount,
  AVG(trip_distance) as avg_distance
FROM
 live.bronze_taxi_trips
GROUP BY week
ORDER BY week ASC;

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW gold_top_n
AS SELECT
  silver_weekly_stats.week,
  ROUND(avg_amount,2) as avg_amount, 
  ROUND(avg_distance,3) as avg_distance,
  fare_amount, trip_distance, zip 
FROM live.silver_flagged_rides
LEFT JOIN live.silver_weekly_stats ON silver_weekly_stats.week = silver_flagged_rides.week
ORDER BY fare_amount DESC
LIMIT 3;